In [18]:
import copy
import keras
import numpy as np

In [19]:
model = keras.models.load_model('model_b256_e30.h5')

In [20]:
def norm(a):
    return (a/9)-.5
    
def denorm(a):
    return (a+.5)*9

In [21]:
def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = copy.copy(sample)
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

In [34]:
def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)

        print("-----------------------------")
        print("Predict:", i)
        print(pred)
        
        true = labels[i].reshape((9,9))+1
        
        print("-----------------------------")
        print("Solution:", i)
        print(pred)

        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

In [32]:
import pandas as pd
file = 'filesud.csv'
df = pd.read_csv(file)
#print(df.describe())
#print(df.head())

import numpy as np
size = 10
x = np.zeros((size,9,9),dtype='uint8')
y = np.zeros((size,9,9),dtype='uint8')

df = df.values

def process_matrix(series):
    series = str(series)
    np_small = np.array([int (x) for x in list(series)])
    return np_small.reshape((9,9))

for i in range(size):
    quiz = df[i,0]
    soln = df[i,1]
    x[i] = process_matrix(quiz)
    y[i] = process_matrix(soln)
    
#print(x[2])
#print(y[2])

In [36]:
test_accuracy(x, y)

-----------------------------
Predict: 0
[[1 2 5 8 2 5 6 6 3]
 [7 1 7 7 2 6 2 1 5]
 [1 7 1 7 7 3 8 1 7]
 [3 3 6 7 1 2 3 3 5]
 [2 3 7 2 2 7 6 1 2]
 [1 6 7 7 6 3 7 1 2]
 [9 7 7 2 2 2 2 2 1]
 [6 1 7 3 7 7 6 3 6]
 [3 3 3 6 5 7 3 5 1]]
-----------------------------
Solution: 0
[[1 2 5 8 2 5 6 6 3]
 [7 1 7 7 2 6 2 1 5]
 [1 7 1 7 7 3 8 1 7]
 [3 3 6 7 1 2 3 3 5]
 [2 3 7 2 2 7 6 1 2]
 [1 6 7 7 6 3 7 1 2]
 [9 7 7 2 2 2 2 2 1]
 [6 1 7 3 7 7 6 3 6]
 [3 3 3 6 5 7 3 5 1]]
-----------------------------
Predict: 1
[[1 3 7 8 7 5 7 3 3]
 [1 8 6 1 2 3 7 1 5]
 [1 7 1 2 3 6 7 3 7]
 [7 3 1 8 1 2 7 5 9]
 [2 1 7 6 7 7 7 1 2]
 [3 7 7 6 5 7 7 1 7]
 [1 6 7 8 2 2 2 7 3]
 [3 1 1 7 6 6 6 3 3]
 [1 2 3 3 3 3 1 5 6]]
-----------------------------
Solution: 1
[[1 3 7 8 7 5 7 3 3]
 [1 8 6 1 2 3 7 1 5]
 [1 7 1 2 3 6 7 3 7]
 [7 3 1 8 1 2 7 5 9]
 [2 1 7 6 7 7 7 1 2]
 [3 7 7 6 5 7 7 1 7]
 [1 6 7 8 2 2 2 7 3]
 [3 1 1 7 6 6 6 3 3]
 [1 2 3 3 3 3 1 5 6]]
-----------------------------
Predict: 2
[[3 5 1 2 3 5 7 3 3]
 [1 1 6 7 2 